In [118]:
import pandas as pd
import numpy as np
from finance_byu.summarize import summary
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore

In [ ]:
# Read in and merge crsp daily and benchmark data
df = pd.read_csv("subset.csv", index_col=0)
bmk = pd.read_csv("bmk.csv")
df = df.merge(bmk, on='caldt', how='left')

df = df.drop_duplicates()

df

,permno,caldt,shrcd,exchcd,ticker,prc,ret,shrout,mkt
0,10001,2010-01-04,11.0,2.0,EGAS,10.25000,-0.004854,4361.0,NaN
1,10001,2010-01-05,11.0,2.0,EGAS,10.19000,-0.005854,4361.0,0.002561
2,10001,2010-01-06,11.0,2.0,EGAS,10.31000,0.011776,4361.0,0.001954
3,10001,2010-01-07,11.0,2.0,EGAS,9.96000,-0.033948,4361.0,0.003449
4,10001,2010-01-08,11.0,2.0,EGAS,10.34000,0.038153,4361.0,0.003587
...,...,...,...,...,...,...,...,...,...
13458532,93436,2023-12-22,11.0,3.0,TSLA,252.53999,-0.007701,3178921.0,0.002534
13458533,93436,2023-12-26,11.0,3.0,TSLA,256.60999,0.016116,3178921.0,0.004689
13458534,93436,2023-12-27,11.0,3.0,TSLA,261.44000,0.018822,3178921.0,0.002297
13458535,93436,2023-12-28,11.0,3.0,TSLA,253.17999,-0.031594,3178921.0,-0.000218


In [120]:
# Compute residuals to mkt factor (IWV daily returns)
window = 20

grouped = df.groupby('permno')

roll_mean_ret = grouped['ret'].rolling(window).mean().reset_index(drop=True)
roll_mean_mkt = grouped['mkt'].rolling(window).mean().reset_index(drop=True)

roll_cov = grouped[['mkt', 'ret']].apply(
    lambda x: x['ret'].rolling(window).cov(x['mkt'])
).reset_index(drop=True)

roll_var = grouped['mkt'].rolling(window).var().reset_index(drop=True)

beta = roll_cov / roll_var

alpha = roll_mean_ret - beta * roll_mean_mkt

fitted = alpha + beta * df['mkt']

residuals = df['ret'] - fitted

df['res'] = residuals

df

,permno,caldt,shrcd,exchcd,ticker,prc,ret,shrout,mkt,res
0,10001,2010-01-04,11.0,2.0,EGAS,10.25000,-0.004854,4361.0,NaN,NaN
1,10001,2010-01-05,11.0,2.0,EGAS,10.19000,-0.005854,4361.0,0.002561,NaN
2,10001,2010-01-06,11.0,2.0,EGAS,10.31000,0.011776,4361.0,0.001954,NaN
3,10001,2010-01-07,11.0,2.0,EGAS,9.96000,-0.033948,4361.0,0.003449,NaN
4,10001,2010-01-08,11.0,2.0,EGAS,10.34000,0.038153,4361.0,0.003587,NaN
...,...,...,...,...,...,...,...,...,...,...
13458532,93436,2023-12-22,11.0,3.0,TSLA,252.53999,-0.007701,3178921.0,0.002534,-0.011243
13458533,93436,2023-12-26,11.0,3.0,TSLA,256.60999,0.016116,3178921.0,0.004689,0.009250
13458534,93436,2023-12-27,11.0,3.0,TSLA,261.44000,0.018822,3178921.0,0.002297,0.016981
13458535,93436,2023-12-28,11.0,3.0,TSLA,253.17999,-0.031594,3178921.0,-0.000218,-0.028194


In [121]:
# Cleaning

df = df[['permno', 'ticker', 'caldt', 'prc', 'ret', 'mkt', 'res']].copy()

df['caldt'] = pd.to_datetime(df['caldt'])

df['year'] = df['caldt'].dt.year
df['month'] = df['caldt'].dt.month

df.head()

,permno,ticker,caldt,prc,ret,mkt,res,year,month
0,10001,EGAS,2010-01-04,10.25,-0.004854,NaN,NaN,2010,1
1,10001,EGAS,2010-01-05,10.19,-0.005854,0.002561,NaN,2010,1
2,10001,EGAS,2010-01-06,10.31,0.011776,0.001954,NaN,2010,1
3,10001,EGAS,2010-01-07,9.96,-0.033948,0.003449,NaN,2010,1
4,10001,EGAS,2010-01-08,10.34,0.038153,0.003587,NaN,2010,1


In [122]:
# Calculate momentum signal

# Log Returns
df['logret'] = np.log1p(df['ret'])

# Momentum from t-6 to t-1
df['mom'] = df.groupby('permno')['logret'].rolling(window*11,window*11).sum().reset_index(drop=True)
df['mom'] = df.groupby('permno')['mom'].shift(window)

df = df.dropna().reset_index(drop=True)

df

,permno,ticker,caldt,prc,ret,mkt,res,year,month,logret,mom
0,10001,EGAS,2010-12-14,10.42000,-0.005725,0.000404,-0.007673,2010,12,-0.005741,0.019477
1,10001,EGAS,2010-12-15,10.51000,0.008637,-0.004315,0.006184,2010,12,0.008600,0.021347
2,10001,EGAS,2010-12-16,10.41000,-0.009515,0.006229,-0.011650,2010,12,-0.009561,0.030214
3,10001,EGAS,2010-12-17,10.40000,-0.000961,0.002153,-0.003056,2010,12,-0.000961,0.017509
4,10001,EGAS,2010-12-20,10.37000,-0.002885,0.002014,-0.005329,2010,12,-0.002889,0.042017
...,...,...,...,...,...,...,...,...,...,...,...
11631186,93436,TSLA,2023-12-22,252.53999,-0.007701,0.002534,-0.011243,2023,12,-0.007731,0.683641
11631187,93436,TSLA,2023-12-26,256.60999,0.016116,0.004689,0.009250,2023,12,0.015988,0.650205
11631188,93436,TSLA,2023-12-27,261.44000,0.018822,0.002297,0.016981,2023,12,0.018647,0.691532
11631189,93436,TSLA,2023-12-28,253.17999,-0.031594,-0.000218,-0.028194,2023,12,-0.032104,0.690453


In [ ]:
# Compute Alphas

df['ic'] = (
    df.groupby('permno')[['mom', 'res']]
    .apply(lambda group: group['mom'].rolling(22, 22).corr(group['res']))
    .reset_index(level=0, drop=True)
)

df['res_vol'] = df.groupby('permno')['res'].rolling(22,22).std().reset_index(drop=True)

df['z_mom'] = df.groupby('caldt')['mom'].apply(zscore).reset_index(drop=True)

df['alpha'] = df['ic'] * df['res_vol'] * df['z_mom']

df

,permno,ticker,caldt,prc,ret,mkt,res,year,month,logret,mom,ic,res_vol,z_mom,alpha
0,10001,EGAS,2010-12-14,10.42000,-0.005725,0.000404,-0.007673,2010,12,-0.005741,0.019477,NaN,NaN,-0.147735,NaN
1,10001,EGAS,2010-12-15,10.51000,0.008637,-0.004315,0.006184,2010,12,0.008600,0.021347,NaN,NaN,-0.474147,NaN
2,10001,EGAS,2010-12-16,10.41000,-0.009515,0.006229,-0.011650,2010,12,-0.009561,0.030214,NaN,NaN,-1.244375,NaN
3,10001,EGAS,2010-12-17,10.40000,-0.000961,0.002153,-0.003056,2010,12,-0.000961,0.017509,NaN,NaN,-0.006753,NaN
4,10001,EGAS,2010-12-20,10.37000,-0.002885,0.002014,-0.005329,2010,12,-0.002889,0.042017,NaN,NaN,2.719760,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11631186,93436,TSLA,2023-12-22,252.53999,-0.007701,0.002534,-0.011243,2023,12,-0.007731,0.683641,0.038838,0.020825,0.608299,0.000492
11631187,93436,TSLA,2023-12-26,256.60999,0.016116,0.004689,0.009250,2023,12,0.015988,0.650205,-0.074504,0.019425,0.329450,-0.000477
11631188,93436,TSLA,2023-12-27,261.44000,0.018822,0.002297,0.016981,2023,12,0.018647,0.691532,-0.026724,0.019668,-0.037672,0.000020
11631189,93436,TSLA,2023-12-28,253.17999,-0.031594,-0.000218,-0.028194,2023,12,-0.032104,0.690453,-0.034562,0.020605,-0.793430,0.000565
